In [ ]:
import logging
import sacrebleu
import pandas as pd
import sys
from simpletransformers.t5 import T5Model, T5Args

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
model_args = T5Args()
model_args.max_length = 512
model_args.length_penalty = 1
model_args.num_beams = 10

In [ ]:
#model = T5Model("mt5", "persiannlp/mt5-base-parsinlu-opus-translation_fa_en", args=model_args)
model = T5Model("t5", "outputs/best_model", args=model_args)

In [ ]:
eval_df = pd.read_csv("data/eval_xWant.tsv", sep="\t").astype(str)

In [ ]:
xWant_truth = eval_df.loc[eval_df["prefix"] == "xWant"]["target_text"].tolist()
to_xWant = eval_df.loc[eval_df["prefix"] == "xWant"]["input_text"].tolist()

In [ ]:
test_set = ["xWant:" + x for x in to_xWant[:100]]
#print(test_set)
xWant_preds = model.predict(to_xWant)
xWant_preds

In [ ]:
xWant_truth

In [ ]:
len(xWant_truth)

In [ ]:
xWant_bleu = sacrebleu.corpus_bleu(xWant_preds, xWant_truth)
print(xWant_bleu.score)

with open('test_xWant.txt', 'w') as f:
        print(xWant_bleu.score, file=f)
        for truth, pred in zip(xWant_truth, xWant_preds):
            print(truth,"--", pred)
            print(truth,"--", pred, file =f)

In [ ]:
print("--------------------------")
print("xWant: ", xWant_bleu.score)

In [ ]:
print("predicting")
print(model.predict(["PersonX go to the bar","PersonX hunts"]))
sys.exit()